In [1]:
import os
os.chdir('C:\\Users\\HBLinks\\anaconda3\\envs\\MLLAB\\LABS\\Lab7\\Lab9')

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input, SimpleRNN
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [3]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [4]:
def create_rnn():
    input_data = Input(shape=(time_steps, num_features))
    rnn_layer1 = SimpleRNN(8, return_sequences=True)(input_data)
    rnn_layer2 = SimpleRNN(20)(rnn_layer1)
    x = Flatten()(rnn_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [5]:
model1 = create_rnn()
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 24, 8)               │             240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 20)                  │             580 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 841 (3.29 KB)

 Trainable params: 841 (3.29 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
tensorflow.keras.utils.plot_model(model1 )

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [8]:
checkpoints = r'C:\\Users\\HBLinks\\anaconda3\\envs\\MLLAB\\LABS\\Lab7\\Lab9\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\\Users\\HBLinks\\anaconda3\\envs\\MLLAB\\LABS\\Lab7\\Lab9'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [9]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]

In [10]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_rnn()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [11]:
import os
path_dataset ='C:\\Users\\HBLinks\\anaconda3\\envs\\MLLAB\\LABS\\Lab7'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

C:\Users\HBLinks\anaconda3\envs\MLLAB\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [12]:
time_steps=24
num_features=21

In [13]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.8727459907531738 sec


In [14]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/10
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0706 - mae: 0.0706 - mape: 51.1448
Epoch 1: val_loss improved from inf to 0.02117, saving model to C:\\Users\\HBLinks\\anaconda3\\envs\\MLLAB\\LABS\\Lab7\\Lab9\\E1-cp-0001-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 33s 11ms/step - loss: 0.0706 - mae: 0.0706 - mape: 51.1483 - val_loss: 0.0212 - val_mae: 0.0212 - val_mape: 9.4274
Epoch 2/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0148 - mae: 0.0148 - mape: 35.2128
Epoch 2: val_loss improved from 0.02117 to 0.01188, saving model to C:\\Users\\HBLinks\\anaconda3\\envs\\MLLAB\\LABS\\Lab7\\Lab9\\E1-cp-0002-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 28s 11ms/step - loss: 0.0148 - mae: 0.0148 - mape: 35.2158 - val_loss: 0.0119 - val_mae: 0.0119 - val_mape: 5.2216
Epoch 3/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0131 - mae: 0.0131 - mape: 4.7289
Epoch 3: val_loss improved from 0.01188 to 0.01078, saving model to C:\\Users\\HBLinks\\anaconda3\\envs\\MLLAB\\LABS\\Lab7\\Lab9\\E1-cp-0003-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 28s 11ms/step - loss: 0.0131 - mae: 0.0131 - mape: 4.7289 - val_loss: 0.0108 - val_mae: 0.0108 - val_mape: 4.8170
Epoch 4/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0120 - mae: 0.0120 - mape: 16.7765
Epoch 4: val_loss improved from 0.01078 to 0.01015, saving model to C:\\Users\\HBLinks\\anaconda3\\envs\\MLLAB\\LABS\\Lab7\\Lab9\\E1-cp-0004-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 29s 11ms/step - loss: 0.0120 - mae: 0.0120 - mape: 16.8257 - val_loss: 0.0102 - val_mae: 0.0102 - val_mape: 4.4255
Epoch 5/10
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0113 - mae: 0.0113 - mape: 360.4883
Epoch 5: val_loss improved from 0.01015 to 0.00977, saving model to C:\\Users\\HBLinks\\anaconda3\\envs\\MLLAB\\LABS\\Lab7\\Lab9\\E1-cp-0005-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 29s 11ms/step - loss: 0.0113 - mae: 0.0113 - mape: 360.2724 - val_loss: 0.0098 - val_mae: 0.0098 - val_mape: 4.4361
Epoch 6/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0107 - mae: 0.0107 - mape: 10.5771
Epoch 6: val_loss did not improve from 0.00977
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 27s 10ms/step - loss: 0.0107 - mae: 0.0107 - mape: 10.5948 - val_loss: 0.0104 - val_mae: 0.0104 - val_mape: 5.3868
Epoch 7/10
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0103 - mae: 0.0103 - mape: 24.6195
Epoch 7: val_loss improved from 0.00977 to 0.00909, saving model to C:\\Users\\HBLinks\\anaconda3\\envs\\MLLAB\\LABS\\Lab7\\Lab9\\E1-cp-0007-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 27s 10ms/step - loss: 0.0103 - mae: 0.0103 - mape: 24.6990 - val_loss: 0.0091 - val_mae: 0.0091 - val_mape: 3.9784
Epoch 8/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0097 - mae: 0.0097 - mape: 38.2854
Epoch 8: val_loss did not improve from 0.00909
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 27s 10ms/step - loss: 0.0097 - mae: 0.0097 - mape: 38.2830 - val_loss: 0.0099 - val_mae: 0.0099 - val_mape: 4.3267
Epoch 9/10
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0094 - mae: 0.0094 - mape: 50.1800
Epoch 9: val_loss did not improve from 0.00909
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 29s 11ms/step - loss: 0.0094 - mae: 0.0094 - mape: 50.2529 - val_loss: 0.0111 - val_mae: 0.0111 - val_mape: 5.2502
Epoch 10/10
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0090 - mae: 0.0090 - mape: 58.5209
Epoch 10: val_loss improved from 0.00909 to 0.00891, saving model to C:\\Users\\HBLinks\\anaconda3\\envs\\MLLAB\\LABS\\Lab7\\Lab9\\E1-cp-0010-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 28s 11ms/step - loss: 0.0090 - mae: 0.0090 - mape: 58.6051 - val_loss: 0.0089 - val_mae: 0.0089 - val_mape: 4.4465


In [22]:

model = load_model(r'C:\Users\HBLinks\anaconda3\envs\MLLAB\LABS\Lab7\Lab9\E1-cp-0010-loss0.01.h5', compile=False)

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step
Mean Absolute Error (MAE): 140.52
Median Absolute Error (MedAE): 117.05
Mean Squared Error (MSE): 32506.86
Root Mean Squared Error (RMSE): 180.3
Mean Absolute Percentage Error (MAPE): 0.98 %
Median Absolute Percentage Error (MDAPE): 0.81 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [23]:
checkpoints = r'C:\Users\HBLinks\anaconda3\envs\MLLAB\LABS\Lab7\Lab9\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'C:\Users\HBLinks\anaconda3\envs\MLLAB\LABS\Lab7\Lab9\E1-cp-0010-loss0.01.h5'
start_epoch= 0

In [24]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)


# construct the set of callbacks
callbacks = [EpochCheckpoint1]
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading C:\Users\HBLinks\anaconda3\envs\MLLAB\LABS\Lab7\Lab9\E1-cp-0010-loss0.01.h5...


TypeError: Could not locate function 'mae'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'keras.metrics', 'class_name': 'function', 'config': 'mae', 'registered_name': 'mae'}

In [19]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

AttributeError: 'str' object has no attribute 'fit'

In [19]:

model = load_model(r'C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab9\E2-cp-0006-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 1s 1ms/step
Mean Absolute Error (MAE): 144.75
Median Absolute Error (MedAE): 113.65
Mean Squared Error (MSE): 36189.68
Root Mean Squared Error (RMSE): 190.24
Mean Absolute Percentage Error (MAPE): 0.98 %
Median Absolute Percentage Error (MDAPE): 0.79 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
